## <span style="color:orange">Data Processor</span> for BitCoin/Bank Secondary Issuances Market Timing Strategy

This sheet starts the process of coding a backtest for a strategy timing the US equity market using the bank secondary issuances as a signal. The first step is to load data on stock returns for BTC and banks and the dates of secondary issuances, and organize it so we can access it as we backtest.

### Data processor required functions

According to the pseudo-code above, the <span style="color:green">Data Processor</span> needs to do the following tasks:
1. Load all the necessary raw data (in constructor)
1. Return an array of unique dates in the raw data (`unique_dates()`)
1. For a given date, return a signal DataFrame containing all the latest signals for the appropriate universe of securities (`signal_df_for_date(date)`)
1. For a given date, return a price DataFrame containing the latest prices for all securities potentially in the portfolio, including those not in the current investable universe (`price_df_for_date(date)`)

In [3]:
import pandas as pd
import datetime as dt
import numpy as np
from pathlib import Path
from constants import holding_period

In [4]:
# Define class here
class SBFDataProcessor():
    
    # Path to where we store the data
    data_folder_path = Path('data') 

    # Minimum share price to open a new position
    min_share_price = 0

    # Start Date
    start_date = pd.to_datetime('2014-09-17')
        
    # Constructor, loads/cleans/merges data as needed
    def __init__(self):
        
        # Load price data: monthly 1926-2022 sample of mkt_rf 
        self.price_df = pd.read_csv(self.data_folder_path / 'yf-prices.csv')
        
        # Parse the yyyyMMdd int dates into DateTime64
        # Based on formatting strings here
        # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

        # Define prices of all stocks and Crypto in universe
        self.price_df['date'] = pd.to_datetime(self.price_df.loc[:,'Date'])
        self.price_df['prc'] = self.price_df['Adj Close']
        self.price_df['security_id'] = self.price_df['Ticker']
        self.price_df['ret_next'] = safe_lead_lag(self.price_df.loc[:,'ret'],self.price_df.loc[:,'security_id'],1)
        self.price_df['ret_holding_period'] = safe_lead_lag(self.price_df.loc[:,'ret'],self.price_df.loc[:,'security_id'], holding_period)
        self.price_df = self.price_df[['date', 'prc', 'security_id', 'ret', 'ret_next', 'ret_holding_period']]
        self.price_df = self.price_df[self.price_df['date'] >= self.start_date]

        # Define all signals in traing strategy
        self.signal_df = pd.read_csv(self.data_folder_path / 'signal-raw.csv')
        self.signal_df['date'] = pd.to_datetime(self.signal_df.loc[:,'Date'])
        self.signal_df['security_id'] = self.signal_df['Ticker']
        self.signal_df['size'] = self.signal_df['Offer Size (M)']
        self.signal_df = self.signal_df[['date', 'security_id', 'size']]
        self.signal_df = self.signal_df[self.signal_df['date'] >= self.start_date]
        
    # Returns an array with the unique dates for which we have loaded data
    # Uses from the price_df since that's how frequency we can update portfolio value
    # Filters all dates in price_df to return only dates for which we have signals as well
    def unique_dates(self):
        price_dates = pd.Series( np.sort(self.price_df.loc[:,'date'].unique()) )
        return price_dates[1:].array
    
    # Returns a filtered version of the passed DataFrame,
    # with all observations deemed too illiquid removed
    # Liquidity requirements:
    #  - price >= $3
    def liquidity_filter(self,df):
        return df.loc[ df.loc[:,'prc'] >= self.min_share_price,:]
    
    # Returns a DataFrame containing one row for all securities in price_df as of date.
    # Columns must include:
    # - 'date': date on which price data observed
    # - 'security_id': a security identifier
    # - 'prc': price on date
    # - 'ret': return from previous date to date
    # Ignores liquidity and future-return availability requirements
    # To be used only for closing decisions and execution decisions
    # Some of the returned stocks cannot be traded
    def price_df_for_date(self,date):
        price_date_df = self.price_df.loc[ self.price_df.loc[:,'date'] == date, :]
        return price_date_df
    
    # Returns a DataFrame where each row is a security in the strategy's universe,
    # Columes must include:
    # - 'date': date on which price data observed
    # - 'security_id': a security identifier
    # - whatever signals the trading rule needs to decide which securities to open new positions in
    #   - In this case, return cshoq, prccq, and ceqq so trading rule can compute B/M ratio
    #
    # Also responsible for applying whatever liquidity filters are wanted to narrow universe,
    # and check that we have future return data (no point in backtesting if we don't know what happens next)
    def signal_df_for_date(self,date):
        # Merge on secruity id
        merged_df = pd.merge(self.price_df, self.signal_df, on = 'security_id', suffixes=['_price', '_signal'])

        # Find all dates of the price is a max of 3 days after date of signal
        merged_df = merged_df[(merged_df['date_price']-merged_df['date_signal']>= pd.Timedelta(days=0)) & (merged_df['date_price']-merged_df['date_signal']<= pd.Timedelta(days=3))]
        
        # Group by security_id and date_signal to find newest date_price after signal
        merged_df = merged_df.sort_values(by=['security_id', 'date_price']).groupby(by=['security_id', 'date_signal']).first().reset_index()

        # Set date to the date of the price (action)
        merged_df['date'] = merged_df['date_price']

        # Apply liquidity filter
        merged_df = self.liquidity_filter(merged_df)

        # Find and add BTC prices to hedge shorting banks
        btc = self.price_df[self.price_df['security_id'] == "BTC-USD"]
        filtered_btc = btc[btc['date'].isin(merged_df['date'].unique())]

        merged_df = pd.concat([merged_df, filtered_btc])

        merged_df = merged_df[ np.isfinite(merged_df['ret_next'])]
        merged_df = merged_df[ np.isfinite(merged_df['ret_holding_period'])]

        merged_df = merged_df[['security_id', 'prc', 'ret', 'ret_next', 'ret_holding_period', 'size', 'date']]
        
        return merged_df[merged_df['date'] == date]

###################################################################
# Helper methods, do not modify
###################################################################

# Function safe_lead_lag returns a new Series with the lead/lagged values
#  but only when a group is the same for the lead/lag
# Inputs:
# - data_series: data we want to lead/lag
# - group_series: grouping we want to be the same for the lead/lag to be value
# requires data_series and group_series already by sorted by group_series
# so that all alike values of group_series are adjacent,
# meaning group_series should look like:
#    g_0
#    g_0
#    g_0
#    g_0
#    g_1
#    g_1
#    g_2
#    g_2 
#    ...
# where g_i indicates the observation is in group i,
# and once the first g_{i+1} appears no more g_i values appear
# 
# lead_lag > 0 returns a data_series with values of data_series lead_lag rows ahead
# as long as group_series remains the same, NaN if group different
# lead_lag < 0 returns a data_series with values of data_series -lead_lag rows behind 
# (same as lead_lag rows ahead) as long as group_series remains the same, NaN if group different
def safe_lead_lag(data_series,group_series,lead_lag): 
    df = pd.DataFrame({ 'data': data_series, 'group': group_series })
    return df.groupby(['group'])['data'].shift(-lead_lag)